# Predicting MOOC grade improvement using a LSTM recurrent neural network

### Importing data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.preprocessing import sequence
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [5]:
db = pd.read_csv("OutputTable.csv")
db.sort(columns=['UserID', 'ProblemID', 'SubmissionNumber'], inplace=True)
db['NVideoAndForum'] = db['NVideoEvents'] + db['NForumEvents']
db = db[db['NVideoAndForum'] > 0]
db['improved'] = (db['GradeDiff'] > 0).astype(int)
db['dummy'] = (db['improved'] < 1 ).astype(int) 
db.fillna(value=0, inplace=True)
#db.replace(to_replace=db['ProblemID'].values, value=np.arange(0,len(db)))

/Users/Thomas/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [6]:
db.set_index('UserID', inplace=True)
for user in db.index.values:
    sub = db.loc[user]
    sub['ProblemID'] = sub['ProblemID'] + hash(user)

/Users/Thomas/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Thomas/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
db.reset_index(inplace=True)
db.set_index(['UserID', 'ProblemID', 'SubmissionNumber'], inplace=True)
db.head()

TimeStamp  \
UserID                           ProblemID           SubmissionNumber               
0009e7ff2e6bad5d80640eeb61cd6e0d 2695735791881420852 13                1397499746   
                                                     14                1397502429   
                                 2695735791881420855 5                 1397596602   
                                                     9                 1397596892   
                                                     10                1397597189   

                                                                       TimeSinceLast  \
UserID                           ProblemID           SubmissionNumber                  
0009e7ff2e6bad5d80640eeb61cd6e0d 2695735791881420852 13                        529.0   
                                                     14                       2683.0   
                                 2695735791881420855 5                         233.0   
                                                     9                          97.0   
                                                     10                        297.0   

                                                                           Grade  \
UserID                           ProblemID           SubmissionNumber              
0009e7ff2e6bad5d80640eeb61cd6e0d 2695735791881420852 13                88.888889   
                                                     14                77.777778   
                                 2695735791881420855 5                 80.000000   
                                                     9                 86.666667   
                                                     10                86.666667   

                                                                       GradeDiff  \
UserID                           ProblemID           SubmissionNumber              
0009e7ff2e6bad5d80640eeb61cd6e0d 2695735791881420852 13                11.111111   
                                                     14               -11.111111   
                                 2695735791881420855 5                  0.000000   
                                                     9                  0.000000   
                                                     10                 0.000000   

                                                                       NVideoEvents  \
UserID                           ProblemID           SubmissionNumber                 
0009e7ff2e6bad5d80640eeb61cd6e0d 2695735791881420852 13                         0.0   
                                                     14                         0.0   
                                 2695735791881420855 5                          0.0   
                                                     9                          0.0   
                                                     10                         0.0   

                                                                       NForumEvents  \
UserID                           ProblemID           SubmissionNumber                 
0009e7ff2e6bad5d80640eeb61cd6e0d 2695735791881420852 13                         3.0   
                                                     14                        27.0   
                                 2695735791881420855 5                          6.0   
                                                     9                          1.0   
                                                     10                         2.0   

                                                                       NumberOfVideoInteractions  \
UserID                           ProblemID           SubmissionNumber                              
0009e7ff2e6bad5d80640eeb61cd6e0d 2695735791881420852 13                                      0.0   
                                                     14                                      0.0   
                                 2695735791881420855 5       

In [8]:
db.drop(['TimeStamp', 'Grade', 'GradeDiff'], axis=1, inplace=True)
db

TimeSinceLast  \
UserID                           ProblemID            SubmissionNumber                  
0009e7ff2e6bad5d80640eeb61cd6e0d  2695735791881420852 13                        529.0   
                                                      14                       2683.0   
                                  2695735791881420855 5                         233.0   
                                                      9                          97.0   
                                                      10                        297.0   
                                                      11                      46568.0   
                                                      22                        320.0   
0020dea356e254c45d9d4ffe68229b3b  5681382887977460174 1                         436.0   
                                  5681382887977460180 1                       23045.0   
                                  5681382887977460181 3                         499.0   
                                  5681382887977460186 6                         416.0   
0123f0d5ea6362ba71ac18b22583d71d -2079615109846755422 2                        3840.0   
                                                      3                        2450.0   
                                                      4                       69276.0   
                                                      5                       10540.0   
                                                      7                        3425.0   
                                                      8                        1702.0   
                                                      9                         549.0   
                                                      10                        422.0   
                                 -2079615109846755420 4                        6548.0   
                                                      7                         392.0   
                                                      9                         932.0   
                                                      10                        160.0   
                                                      11                        552.0   
                                                      15                      72049.0   
                                                      17                       3525.0   
                                                      20                       6182.0   
                                                      33                      80607.0   
                                                      34                     533164.0   
                                                      35                      57809.0   
...                                                                               ...   
fec803e81a96c3fdc65dfa6cc00e0cbb  2065393602026241647 3                       66877.0   
                                                      4                        1185.0   
                                                      12                       1404.0   
                                                      14                       1075.0   
                                                      15                     270160.0   
                                  2065393602026241649 3                        3540.0   
                                                      6                        1137.0   
                                                      7                      232456.0   
                                                      9                        2860.0   
                                                      13                       7159.0   
                                                      15                        691.0   
                                                      18                      72256.0   
                                                      19           

In [9]:
max_submission = db.index.levshape[2]
max_submission

99

In [61]:
a3d = np.array(list(db.groupby(level=1).apply(pd.DataFrame.as_matrix)))
X_train = sequence.pad_sequences(a3d, maxlen=max_submission, padding='post')
X_train.shape

(1907, 99, 16)

In [62]:
y_train = X_train[:,:,14:17]
X_train = X_train[:,:,:14]

In [63]:
X_train = X_train.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
for i in range(0,X_train.shape[0]):
    X_train[i,:,:] = scaler.fit_transform(X_train[i,:,:])

### Splitting into training and test set

In [20]:
users = db['UserID'].unique()
np.random.seed(1234)
np.random.shuffle(users)
training_users = users[:round(len(users)*0.9)]
db_train = db[db.UserID.isin(training_users)]
db_test = db[~db.UserID.isin(training_users)]

In [21]:
train_subset = db_train.drop(['UserID', 'Grade', 'GradeDiff', 'improved'], axis=1)
test_subset = db_test.drop(['UserID', 'Grade', 'GradeDiff', 'improved'], axis=1)
X_train = train_subset.values
y_train = np_utils.to_categorical(db_train.improved.values)
X_test = test_subset.values
y_test = np_utils.to_categorical(db_test.improved.values)

In [22]:
X_train = X_train.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
#X_test = scaler.fit_transform(X_test)

### LSTM network

In [23]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [24]:
X_train = X_train[0:300,:,:]
y_train = y_train[0:300]

In [29]:
y_train.shape

(300, 2)

In [181]:
x = np.asarray(
[
    [
        [0,0,1],
        [0,0,1],
        [0,0,1],
        #[0,0,1],
    ],
    [
        [1,1,0],
        [0,1,0],
        [1,1,0],
        [0,0,1],
    ],
])
x.shape

(2,)

In [157]:
x.shape

(2,)

In [103]:
y = np.asarray(
[
    [
        [0,1],
        [0,1],
        [1,0],
        [1,0],
    ],
    [
        [0,1],
        [0,1],
        [1,0],
        [1,0],
    ],
])
y.shape

(2, 4, 2)

In [25]:
sum(y_train[:,0]*100)/300

67.0

In [26]:
X_train.shape

(300, 1, 17)

(User*problem, submission, feature)

In [65]:
from keras.layers.core import Masking, TimeDistributedDense


In [68]:
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(99, 14)))
model.add(LSTM(50, return_sequences=True, activation='relu'))
#model.add(TimeDistributed(Dense(30, activation='sigmoid')))
model.add(TimeDistributedDense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/Users/Thomas/anaconda/envs/tensorflow/lib/python3.5/site-packages/keras/layers/core.py:1147: UserWarning: TimeDistributedDense is deprecated, please use TimeDistributed(Dense(...)) instead.
  warnings.warn('TimeDistributedDense is deprecated, '


In [71]:
model.fit(X_train,y_train,batch_size=100, nb_epoch=100)

Epoch 1/100
1907/1907 [==============================] - 11s - loss: 0.6661 - acc: 0.5043    
Epoch 2/100
1907/1907 [==============================] - 11s - loss: 0.6613 - acc: 0.5041    
Epoch 3/100
1907/1907 [==============================] - 11s - loss: 0.6558 - acc: 0.5041    
Epoch 4/100
1907/1907 [==============================] - 11s - loss: 0.6551 - acc: 0.5041    
Epoch 5/100
1907/1907 [==============================] - 11s - loss: 0.6534 - acc: 0.5041    
Epoch 6/100
1907/1907 [==============================] - 11s - loss: 0.6516 - acc: 0.5041    
Epoch 7/100
1907/1907 [==============================] - 12s - loss: 0.6500 - acc: 0.5041    
Epoch 8/100
1907/1907 [==============================] - 11s - loss: 0.6489 - acc: 0.5041    
Epoch 9/100
1907/1907 [==============================] - 11s - loss: 0.6474 - acc: 0.5041    
Epoch 10/100
1907/1907 [==============================] - 11s - loss: 0.6468 - acc: 0.5041    
Epoch 11/100
1907/1907 [==============================] - 1

In [27]:
model = Sequential()
model.add(LSTM(output_dim=50, return_sequences=True, activation='relu', statefull=True, batch_input_size=(300,1,17))
model.add(TimeDistributed(Dense(30, activation='sigmoid')))
model.add(TimeDistributed(Dense(2, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

          
model.fit(X_train, y_train, nb_epoch=100, batch_size=1, verbose=2)

Epoch 1/100
6s - loss: 0.6734 - acc: 0.6083
Epoch 2/100
5s - loss: 0.6371 - acc: 0.6700
Epoch 3/100
5s - loss: 0.6367 - acc: 0.6700
Epoch 4/100
5s - loss: 0.6376 - acc: 0.6700
Epoch 5/100
5s - loss: 0.6338 - acc: 0.6700
Epoch 6/100
5s - loss: 0.6325 - acc: 0.6700
Epoch 7/100
5s - loss: 0.6315 - acc: 0.6700
Epoch 8/100
5s - loss: 0.6286 - acc: 0.6700
Epoch 9/100
5s - loss: 0.6292 - acc: 0.6700
Epoch 10/100
5s - loss: 0.6246 - acc: 0.6700
Epoch 11/100
5s - loss: 0.6202 - acc: 0.6700
Epoch 12/100
5s - loss: 0.6268 - acc: 0.6700
Epoch 13/100
5s - loss: 0.6234 - acc: 0.6700
Epoch 14/100
5s - loss: 0.6187 - acc: 0.6700
Epoch 15/100
5s - loss: 0.6164 - acc: 0.6700
Epoch 16/100
5s - loss: 0.6165 - acc: 0.6700
Epoch 17/100
5s - loss: 0.6139 - acc: 0.6700
Epoch 18/100
5s - loss: 0.6147 - acc: 0.6700
Epoch 19/100
5s - loss: 0.6113 - acc: 0.6700
Epoch 20/100
5s - loss: 0.6106 - acc: 0.6700
Epoch 21/100
5s - loss: 0.6089 - acc: 0.6717
Epoch 22/100
5s - loss: 0.6097 - acc: 0.6717
Epoch 23/100
5s - l

KeyboardInterrupt: 

In [72]:
from collections import Counter

In [82]:
a = [1, 1, 1 ,2, 2, 3, 4, 5]
c = Counter(a)
sum(v for v in c.values() if v>1)

5

In [28]:
scores = model.evaluate(X_train, y_train, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))
preds = model.predict_classes(X_test, batch_size=1)

Model Accuracy: 68.83%
487/487 [==============================] - 1s     


In [85]:
dicti = {'EventType': ['Video.Load', 'Video.SpeedChange', 'Video.Seek', 'Video.Seek', 'Video.Stalled', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Load', 'Video.SpeedChange', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Pause', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Load', 'Video.SpeedChange', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Seek', 'Video.Pause']}

In [114]:
lista = list(dicti.values())[0]
indexes = [i for i, j in enumerate(lista) if j == 'Video.Load']
indexes

[0, 15, 27]